## Multimodal embedding.

Feedback : shins777@gmail.com. 

구글은 이미지해석, 생성 등의 멀티모달이 가능한 모델을 제공합니다.
이런 처리를 위해서 멀티모달 임베딩이 필요하며, 이 Colab 은 멀티모달 임베딩을 할 수 있는 예제입니다.

* https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-multimodal-embeddings?hl=ko
* https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings


In [10]:
%pip install --upgrade --quiet google-cloud-aiplatform scikit-learn

Note: you may need to restart the kernel to use updated packages.


### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
# from google.colab import auth
# auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [2]:
project="ai-hangsik"
location = "us-central1"
model_name="multimodalembedding@001"

### Vertex AI 초기화

In [4]:
from google.cloud import aiplatform
aiplatform.init(project="ai-hangsik", location = location )

### Embedding

In [5]:

from absl import app
from absl import flags
import base64
# Need to do pip install google-cloud-aiplatform for the following two imports.
# Also run: gcloud auth application-default login.
from google.cloud import aiplatform
from google.protobuf import struct_pb2
import sys
import time
import typing

class EmbeddingResponse(typing.NamedTuple):
  text_embedding: typing.Sequence[float]
  image_embedding: typing.Sequence[float]


#### Embedding function

In [12]:
def get_embedding(text : str = None, image_bytes : bytes = None)->EmbeddingResponse:

  api_regional_endpoint = "us-central1-aiplatform.googleapis.com"
  client_options = {"api_endpoint": api_regional_endpoint}
  # Initialize client that will be used to create and send requests.
  # This client only needs to be created once, and can be reused for multiple requests.
  client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

  if not text and not image_bytes:
    raise ValueError('At least one of text or image_bytes must be specified.')

  instance = struct_pb2.Struct()
  if text:
    instance.fields['text'].string_value = text

  if image_bytes:
    encoded_content = base64.b64encode(image_bytes).decode("utf-8")
    image_struct = instance.fields['image'].struct_value
    image_struct.fields['bytesBase64Encoded'].string_value = encoded_content

  instances = [instance]
  endpoint = (f"projects/{project}/locations/{location}/publishers/google/models/{model_name}")
  response = client.predict(endpoint=endpoint, instances=instances)

  text_embedding = None
  if text:
    text_emb_value = response.predictions[0]['textEmbedding']
    text_embedding = [v for v in text_emb_value]

  image_embedding = None
  if image_bytes:
    image_emb_value = response.predictions[0]['imageEmbedding']
    image_embedding = [v for v in image_emb_value]

  return EmbeddingResponse(
    text_embedding=text_embedding,
    image_embedding=image_embedding)

#### Image Embedding

In [13]:
image_file = "../../contents/images/cat_bike.png"

image_file_contents = None

with open(image_file, "rb") as f:
    image_file_contents = f.read()

response = get_embedding(text=None, image_bytes=image_file_contents)

embeded_image =response.image_embedding

#### Text Embedding & Similarity

In [14]:
text_value = "고양이가 오토바이를 탑니다."
text_value = "고양이가 오토바이를 탑니다. 도로 주위 나무는 단풍이 들었습니다."
# text_value = "파란눈의 고양이가 오토바이를 탑니다. 도로 주위 나무는 단풍이 들었습니다."
# text_value = "파란 하늘날 파란눈의 고양이가 오토바이를 탑니다. 도로 주위 나무는 단풍이 들었습니다."
# text_value = "파란 하늘날 파란눈의 고양이가 오토바이를 탑니다. 도로위에 흰줄이 있고 주위 나무는 단풍이 들었습니다."

response = get_embedding(text=text_value, image_bytes=None)

embeded_text = response.text_embedding

from sklearn.metrics.pairwise import cosine_similarity

similarity_ratio = cosine_similarity([embeded_text], [embeded_image])[0][0]
print(similarity_ratio)




0.17430823067786372
